In [ ]:
"""
AI-Powered Sports Broadcast Tracking Pipeline
=============================================
An advanced computer vision pipeline leveraging YOLOv8 for semantic 
object detection and ByteTrack for persistent multi-object tracking (MOT). 

This system renders professional broadcast-style visual overlays, including 
dynamic bounding boxes and fading historical trajectory trails. It is 
engineered for cloud execution environments (e.g., Google Colab), featuring 
automated Google Drive integration for seamless, high-volume video I/O.

Author: Raj Antala
"""

import cv2
import numpy as np
import os
import shutil
from collections import defaultdict
from ultralytics import YOLO

# Safe import for Google Colab environment
try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# ============================================================================
# CONFIGURATION
# ============================================================================
INPUT_VIDEO_PATH = "sports_video.mp4"  # Make sure this matches your uploaded file
OUTPUT_VIDEO_PATH = "yolo_basketball_tracking.mp4"
TRAIL_LENGTH = 30

def process_deep_tracking(input_path, output_path):
    if not os.path.exists(input_path):
        print(f"Error: Could not find video at '{input_path}'")
        return

    print("Loading YOLOv8 model...")
    # 'yolov8m.pt' is the medium model (Great balance of accuracy and speed)
    model = YOLO('yolov8m.pt')

    cap = cv2.VideoCapture(input_path)
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read the first frame.")
        return

    h, w = frame.shape[:2]
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    if fps == 0: fps = 30
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

    # Dictionary to store the historical center points for the broadcast trails
    track_history = defaultdict(lambda: [])

    print("Starting Deep Learning Tracking...")
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        frame_count += 1
        if frame_count % 50 == 0:
            print(f"Processed {frame_count} frames...")

        # Run YOLOv8 tracking using ByteTrack
        # classes=[0, 32] tells YOLO to ONLY look for 'person' (0) and 'sports ball' (32)
        results = model.track(frame, persist=True, classes=[0, 32], tracker="bytetrack.yaml", verbose=False)

        # Ensure we actually detected something before trying to draw
        if results[0].boxes is not None and results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            track_ids = results[0].boxes.id.cpu().numpy().astype(int)
            clss = results[0].boxes.cls.cpu().numpy().astype(int)

            for box, track_id, cls in zip(boxes, track_ids, clss):
                x1, y1, x2, y2 = map(int, box)

                # Calculate the center bottom for players (looks better for foot trails), and true center for the ball
                center_x = int((x1 + x2) / 2)
                center_y = int(y2) if cls == 0 else int((y1 + y2) / 2)

                # Set colors: Green for players, Orange for the ball
                color = (0, 255, 0) if cls == 0 else (0, 165, 255)
                label = f"Player ID:{track_id}" if cls == 0 else "Ball"

                # Draw the bounding box and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                # Manage the Broadcast Trail History
                track = track_history[track_id]
                track.append((center_x, center_y))
                if len(track) > TRAIL_LENGTH:
                    track.pop(0)

                # Draw the fading trail line
                for i in range(1, len(track)):
                    thickness = int(np.sqrt(64 / float(i + 1)) * 1.5)
                    cv2.line(frame, track[i - 1], track[i], color, thickness)

        # Dashboard UI Overlay
        cv2.rectangle(frame, (10, 10), (550, 60), (0, 0, 0), -1)
        cv2.putText(frame, "AI TRACKING: YOLOv8 + ByteTrack", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        out.write(frame)

    cap.release()
    out.release()
    print(f"\nSuccess! Total frames processed: {frame_count}")
    print(f"Video temporarily saved to Colab as: {output_path}")

    # ========================================================================
    # SAFE DOWNLOAD: COPY TO GOOGLE DRIVE
    # ========================================================================
    if IN_COLAB:
        print("\nMounting Google Drive to save the video safely...")
        drive.mount('/content/drive')

        drive_path = f"/content/drive/MyDrive/{output_path}"
        shutil.copy(output_path, drive_path)
        print(f"✅ Video successfully backed up to your Google Drive at: {drive_path}")
        print("You can now open drive.google.com to watch or download it!")

if __name__ == "__main__":
    process_deep_tracking(INPUT_VIDEO_PATH, OUTPUT_VIDEO_PATH)

Loading YOLOv8 model...
Starting Deep Learning Tracking...
Processed 50 frames...
Processed 100 frames...
Processed 150 frames...
Processed 200 frames...
Processed 250 frames...
Processed 300 frames...
Processed 350 frames...
Processed 400 frames...

Success! Total frames processed: 433
Video temporarily saved to Colab as: yolo_basketball_tracking.mp4

Mounting Google Drive to save the video safely...
Mounted at /content/drive
✅ Video successfully backed up to your Google Drive at: /content/drive/MyDrive/yolo_basketball_tracking.mp4
You can now open drive.google.com to watch or download it!
